In [1]:
import numpy as np
import pandas as pd

##

##variables


In [42]:
density = 1.22
gravity = 9.81
area = np.array([0.5, 0.6, 0.7]);
cdRef = np.array([0.9, 1.0,1.1,1.2,1.3]);
mass = np.random.randint(130,161, size=50)
windSpeed = np.random.randint(int(1/3.6),int(20/3.6), size=50)

speed = np.random.randint(int(10/3.6),int(120/3.6), size=200001)
rollingCd = 0.015

yawRange =  np.random.randint(1,91, size=90)

## YAW vs tangential drag industry standard mapping

yaw = [
    [0, 0],
    [5  , 0.020],
    [10 , 0.042],
    [15 , 0.065],
    [20 , 0.085],
    [25 , 0.100],
    [30 , 0.110],
    [35 , 0.115],
    [40 , 0.117],
    [45 , 0.118],
    [50 , 0.117],
    [55 , 0.114],
    [60 , 0.110],
    [65 , 0.105],
    [70 , 0.098],
    [75 , 0.090],
    [80 , 0.080],
    [85 , 0.068],
    [90 , 0.050]
]

tangentialSumWithStep = 5;
for angle in yaw:
    tangentialSumWithStep = tangentialSumWithStep + angle[1]*5;


# aerodynamics drag

Training_data = [];
iteration = 0

for vehicleSpeed in speed :
    iteration = iteration + 1;
    ####################################   reference  drag section start ################################# 
    # refrecne drag
    areaChoose = np.random.choice(area)
    CdChoose = np.random.choice(cdRef)
    massChoose = np.random.choice(mass)
    aeroDrafRef = (density * vehicleSpeed * vehicleSpeed * areaChoose * CdChoose)/2;
    ## reference rolling drag
    rollingDragref = massChoose * gravity * rollingCd
    
    netRefrenceDrag = aeroDrafRef + rollingDragref;
    ####################################   reference  drag section ends ################################# 
    ################################################################################################
    ####################################   actual  drag section start ################################# 
    
    cdstar = 0;
    net_speed = 0;
    choosendWhindSpeed = np.random.choice(windSpeed);
    choosenYaw = np.random.choice(yawRange);
    choosenYaw = (3.14/180) * choosenYaw;
    net_speed = np.sqrt(choosendWhindSpeed*choosendWhindSpeed + vehicleSpeed*vehicleSpeed + 2*vehicleSpeed*choosendWhindSpeed*np.cos(choosenYaw));
    
    speed_factor = (net_speed/vehicleSpeed) * (net_speed/vehicleSpeed);
    cdstar = (tangentialSumWithStep * speed_factor)/3.14;
    
    actuallAeroDrag = (density * net_speed * net_speed * areaChoose * cdstar)/2;
    
    netActualDrag = actuallAeroDrag + rollingDragref;
    
    drag_ratio = netActualDrag / netRefrenceDrag;
    k = 0.1;
    recommend_speed = vehicleSpeed * (1 - k*(drag_ratio - 1));
    
    ####################################   actual  drag section end ################################# 
    ################################################################################################
    ####################################  tabular values calculated ################################# 
    speed = vehicleSpeed;
    wind = choosendWhindSpeed;
    yawAngle = (choosenYaw*180)/3.14;
    netSpeedFacing = net_speed;
    cdStarUsed = cdstar;
    Aerodynamic_drag_reference = aeroDrafRef;
    Rolling_drag_reference = rollingDragref;
    Net_reference_drag = netRefrenceDrag;
    Aerodynamic_drag_practicle = actuallAeroDrag;
    Net_actual_drag = netActualDrag;
    Drag_ratio = drag_ratio;
    recommended_speed = recommend_speed*3.6;
    sensitivity_used = k;
    density_used = density;
    mass_vehicle_used = massChoose;
    Cd_reference_used = CdChoose;
    Frontal_area_of_vehicle_used = areaChoose;
    
    Training_data.append([
        speed,
        wind ,
        yawAngle ,
        netSpeedFacing ,
        cdStarUsed ,
        Aerodynamic_drag_reference,
        Rolling_drag_reference ,
        Net_reference_drag ,
        Aerodynamic_drag_practicle,
        Net_actual_drag ,
        Drag_ratio ,
        recommended_speed ,
        sensitivity_used ,
        density_used ,
        mass_vehicle_used ,
        Cd_reference_used ,
        Frontal_area_of_vehicle_used,
    ])
    
    
    

df = pd.DataFrame(Training_data, columns=['speed(m/s)', 'wind(m/s)', 'yawAngle(degrees)', 'netSpeedFacing(m/s)' , 'cdStarUsed', 'Aerodynamic_drag_reference(N)', 'Rolling_drag_reference(N)', 'Net_reference_drag(N)', 'Aerodynamic_drag_practicle(N)' , 'Net_actual_drag(N)', 'Drag_ratio', 'recommended_speed(km/hr)', 'sensitivity_used', 'density_used(kg/m^3)', 'mass_vehicle_used(kg)', 'Cd_reference_used', 'Frontal_area_of_vehicle_used(m^2)' ])

df.to_csv('Training_data_set.csv', index=False)
print(iteration)

200001
